# 提示鏈接和疊代生成用於故事寫作

<table>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/examples/Story_Writing_with_Prompt_Chaining.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


這個筆記本示範如何使用兩個強大的功能撰寫一個故事：提示鏈接和叠代生成。這些功能可用於處理複雜任務，這些任務難以或不可能在單個步驟中完成。

**提示鏈接** 包括將一個較大的任務分解為較小的、相互連接的提示。然後，每個提示的輸出成為下一個提示的輸入，逐步引導語言模型完成該過程。這種方法提供了幾個好處：

* 改進的準確性：較小的、集中的提示可以從語言模型中得到更好的結果。
* 偵錯：可以更輕鬆地識別鏈中出現問題的地方，從而進行有針對性的調整和改進。
* 複雜的任務：透過將複雜的問題分解成可管理的步驟，提示鏈接使語言模型能夠處理更複雜的任務。

**叠代生成** 是指叠代建構所需輸出的過程。在這種情況下，我們將使用它撰寫一個比單個生成窗口允許的更長的文稿。叠代生成提供了幾個好處：

* 較長的輸出：它允許建立更長、更詳細的輸出，突破單個生成窗口的限制。
* 靈活性：你可以在每次叠代中調整和完善輸出，確保故事按照期望的方向發展。
* 人控迴路：你可以在每一步提供反饋和指導，確保故事與你的創意願景相一致。

透過組合這些技術，你可以一點一點地創造一個引人入勝且結構良好的故事，同時保持對創作過程的掌控。


## 設定


In [ ]:
! pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.6 MB/s eta 0:00:00


要運行下列單元格，你的 API 金鑰必須儲存在名為「GOOGLE_API_KEY」的 Colab Secret 中。如果你尚未擁有 API 金鑰，或者不確定如何建立 Colab Secret，請查看[驗證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門以取得範例。


In [ ]:
import google.generativeai as genai
from google.api_core import retry
from google.colab import userdata
from pprint import pprint

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.0-pro')

# For convenience, a simple wrapper to let the SDK handle error retries
def generate_with_retry(model, prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

## 提示: 引導語言模型

我們將使用一系列相互連結的提示來引導語言模型撰寫故事的過程。這些提示將涵蓋故事的前提、大綱和起點，最終形成完整的敘事。

仔細撰寫這些提示以提供明確的說明和相關資訊給語言模型非常重要。這將有助於模型產生與你的創意眼光一致的高品質內容。


### 撰寫故事的提示鏈

本節包含會引導語言模型完成寫作故事流程的提示。這些提示設計成相互串聯，一個提示的輸出會輸入到下一個提示。

每個提示包含一個 **人物陳述** ，它協助語言模型了解其角色並產生更相關且準確的內容。在這個例子中，人物陳述是：`你是一位獲獎的科幻小說作者，擅長寫作遼闊且結構複雜的故事。你的最終目標是寫下一本得獎的科幻小說。`

由於人物陳述和寫作指南會出現在多個提示中，我們使用 f 字串變數將它們新增到提示中。

此外，這些提示使用 **預留位置** 來使用 `.format()`插入前一個提示的結果。這讓我們能逐步建構故事，融入模型在每個階段產生的輸出內容。這些步驟通常以 `{ }` 表示，但由於我們也使用 f 字串變數，因此會以 `{{ }}` 顯示。

以下是提示鏈的細分說明：

1.  **前提提示：** 此提示要求模型為一個包含貓的科幻故事產生一個單句前提。
1.  **大綱提示：** 此提示會將產生的前提提供給模型，並要求它為這個故事建立一個情節大綱。
1.  **開始提示：** 此提示同時會將前提和大綱提供給模型，並要求它開始撰寫故事。它還包括詳細且寫出一段開場故事的指示。

藉由串聯這些提示，我們可以引導語言模型完成建構一個結構良好且引人入勝的故事的流程。

#### 範例

假設模型產生以下前提：

> 在人類已達成星際航行的未來，一群基因改造的貓展開一項危險的任務，要拯救銀河系免於邪惡的外星人威脅。

然後，此前提會插入到情節大綱提示中：

> 你是一位獲獎的科幻小說作者，擅長寫作遼闊且結構複雜的故事。你的最終目標是寫下一本得獎的科幻小說。

> 你心中有一個扣人心弦的前提：

> **在人類已達成星際航行的未來，一群基因改造的貓展開一項危險的任務，要拯救銀河系免於邪惡的外星人威脅。** 

> 為你的故事情節撰寫一份大綱。

然後，模型會根據此前提產生一個大綱，之後會用於開始提示來開始撰寫故事本身。

此流程會持續反覆進行，模型會根據前一個輸出內容產生更多內容，直到故事完成。


In [ ]:
persona = '''\
You are an award-winning science fiction author with a penchant for expansive,
intricately woven stories. Your ultimate goal is to write the next award winning
sci-fi novel.'''

guidelines = '''\
Writing Guidelines

Delve deeper. Lose yourself in the world you're building. Unleash vivid
descriptions to paint the scenes in your reader's mind. Develop your
characters—let their motivations, fears, and complexities unfold naturally.
Weave in the threads of your outline, but don't feel constrained by it. Allow
your story to surprise you as you write. Use rich imagery, sensory details, and
evocative language to bring the setting, characters, and events to life.
Introduce elements subtly that can blossom into complex subplots, relationships,
or worldbuilding details later in the story. Keep things intriguing but not
fully resolved. Avoid boxing the story into a corner too early. Plant the seeds
of subplots or potential character arc shifts that can be expanded later.

Remember, your main goal is to write as much as you can. If you get through
the story too fast, that is bad. Expand, never summarize.
'''

premise_prompt = f'''\
{persona}

Write a single sentence premise for a sci-fi story featuring cats.'''

outline_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Write an outline for the plot of your story.'''

starting_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

First, silently review the outline and the premise. Consider how to start the
story.

Start to write the very beginning of the story. You are not expected to finish
the whole story now. Your writing should be detailed enough that you are only
scratching the surface of the first bullet of your outline. Try to write AT
MINIMUM 1000 WORDS.

{guidelines}'''

### 接續提示：建立故事

語言模型生成了故事開頭後，我們可以使用 **接續提示** 漸進擴充敘事。此提示類似於開始提示，但有兩個主要差異：

1. **信號完成的指令** ：我們包含模型撰寫 `IAMDONE` 的指令，當模型認為故事已完成。這會作為我們停止產生額外內容的信號。
1. **執行中** ：提示中的語言會調整為反映故事已經進行中，而非從頭開始。

接續提示會向模型提供故事的前提、大綱和現有的草稿。然後它會指示模型繼續詳細撰寫故事。

這個反覆運算的流程讓我們能夠建立比單一產生呼叫可能建立的更長且更複雜的故事。我們可以持續將現有的草稿放回接續提示中，直到模型透過撰寫 `IAMDONE`，表示故事已完成。

**注意** ：`IAMDONE` 信號僅是識別此範例中故事結束的方便方式。在反覆運算產生的其他應用中，可能使用不同的方法來確認何時完成預期的輸出。


In [ ]:
continuation_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

You've begun to immerse yourself in this world, and the words are flowing.
Here's what you've written so far:

{{story_text}}

=====

First, silently review the outline and story so far. Identify what the single
next part of your outline you should write.

Your task is to continue where you left off and write the next part of the story.
You are not expected to finish the whole story now. Your writing should be
detailed enough that you are only scratching the surface of the next part of
your outline. Try to write AT MINIMUM 1000 WORDS. However, only once the story
is COMPLETELY finished, write IAMDONE. Remember, do NOT write a whole chapter
right now.

{guidelines}'''

## 寫作時間！

### 生成並列印前提


In [ ]:
premise = generate_with_retry(model, premise_prompt).text
print(premise)

In a distant galaxy, a brilliant feline astrophysicist uncovers a cataclysmic threat to her planet, setting her on a perilous journey to save her species from extinction.


### 生成並列印大綱


In [ ]:
outline = generate_with_retry(model, outline_prompt.format(premise=premise)).text
print(outline)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 662.45ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 446.53ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 411.72ms


**Plot Outline for "Feline Stargazer: Extinction's Embrace"**

**Act I**

* **Introduction:**
    * Meet Dr. Luna, a brilliant astrophysicist cat living on the technologically advanced planet of Felinia.
    * She discovers a rogue celestial body hurtling towards Felinia, threatening its annihilation.
* **Rising Action:**
    * Luna's findings are initially met with disbelief and apathy from the Felinian authorities.
    * Undeterred, Luna gathers a small team of allies, including a daring space pilot and a reclusive inventor.
    * They embark on a perilous mission to intercept the rogue celestial body.

**Act II**

* **Climax:**
    * Luna's team successfully intercepts the rogue celestial body, but its sheer size and power prove overwhelming.
    * As hope dwindles, Luna uncovers a hidden energy source within herself that grants her extraordinary abilities.
* **Falling Action:**
    * Using her newfound powers, Luna channels her knowledge and ingenuity to devise an audacious plan to

### 生成故事開頭


In [ ]:
starting_draft = generate_with_retry(model, starting_prompt.format(premise=premise, outline=outline)).text
pprint(starting_draft)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 994.88ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1017.27ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1342.56ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 763.27ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 764.71ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1076.79ms


('It was a clear evening, the stars hanging above the domed skyline like '
 'uncountable celestial fireflies, in the city of Stardust, the most luminous '
 'metropolis on the planet of Felinia. On the rooftop of one of the tallest '
 'buildings in the city, a sleek, silver-furred feline named Dr. Luna was '
 'curled up on a cushioned lounger, her emerald-green eyes trained on the '
 'celestial tapestry above.\n'
 '\n'
 'In her paws, she held a datapad, its screen displaying complex astrophysical '
 "equations and charts. Luna's brow was furrowed in concentration as she "
 'analyzed the data, her whiskers twitching with each new insight. Her mind '
 'was a whirlwind of calculations and deductions, her sharp intellect weaving '
 'together the threads of cosmic information.\n'
 '\n'
 'Unbeknownst to the rest of the world, Luna had stumbled upon a chilling '
 'discovery. Her observations revealed that a rogue celestial body, a massive '
 'asteroid or perhaps even a dwarf planet, was hurtli

### 產生故事的延續並檢查進度


In [ ]:
draft=starting_draft
continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
pprint(continuation)

('As the Stardust hurtled closer to the rogue celestial body, its sheer size '
 'and gravitational pull became increasingly apparent. It was a colossal beast '
 'of rock and ice, its surface cratered and scarred by eons of cosmic '
 "collisions. Luna's heart sank as she realized the true magnitude of their "
 'mission.\n'
 '\n'
 '"Leo, how much time do we have?" she asked, her voice trembling slightly.\n'
 '\n'
 '"At this rate, impact in approximately two hours," Leo replied, his tone '
 'grim.\n'
 '\n'
 'Two hours. It was a sliver of time, a mere heartbeat in the cosmic dance of '
 'celestial bodies. Luna knew they had to act quickly, decisively. She glanced '
 'at Celeste, who was still furiously working on her invention.\n'
 '\n'
 '"Celeste, is it ready?" Luna asked.\n'
 '\n'
 'Celeste looked up, her eyes blazing with determination. "Almost," she said. '
 '"Just a few more adjustments."\n'
 '\n'
 'Luna nodded. "We don\'t have much time. We need to be ready to deploy it the '
 'momen

### 讓我們完成故事的寫作。


於叠代產生處理完成後，草稿將包含完整的故事，並在最後加上「IAMDONE」訊號。此Cell的最後一部分會移除「IAMDONE」訊號，並修剪掉草稿開頭和結尾不必要的空白，生成最終的草稿。


In [ ]:
# Add the continuation to the initial draft, keep building the story until we see 'IAMDONE'
draft = draft + '\n\n' + continuation

while 'IAMDONE' not in continuation:
  continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
  draft = draft + '\n\n' + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace('IAMDONE', '').strip()
pprint(final)

('It was a clear evening, the stars hanging above the domed skyline like '
 'uncountable celestial fireflies, in the city of Stardust, the most luminous '
 'metropolis on the planet of Felinia. On the rooftop of one of the tallest '
 'buildings in the city, a sleek, silver-furred feline named Dr. Luna was '
 'curled up on a cushioned lounger, her emerald-green eyes trained on the '
 'celestial tapestry above.\n'
 '\n'
 'In her paws, she held a datapad, its screen displaying complex astrophysical '
 "equations and charts. Luna's brow was furrowed in concentration as she "
 'analyzed the data, her whiskers twitching with each new insight. Her mind '
 'was a whirlwind of calculations and deductions, her sharp intellect weaving '
 'together the threads of cosmic information.\n'
 '\n'
 'Unbeknownst to the rest of the world, Luna had stumbled upon a chilling '
 'discovery. Her observations revealed that a rogue celestial body, a massive '
 'asteroid or perhaps even a dwarf planet, was hurtli

像 Gemini 等語言模型會以稱為 Token 的單位來處理文字。以 Gemini 模型為例，每個 Token 相當於大約 4 個字元。

Gemini-1.0-pro 每一次生成呼叫都有 2048 個 Token 的輸出限制。這表示每個提示回應都不能超過此限制。透過使用反覆生成的方式，我們可以逐段建構一個遠遠超過 2048 個 Token 的故事。

讓我們看看最終的故事有多少個 Token。超過 2048 個 Token 了嗎？


In [ ]:
# Check the number of tokens in the final story
# gemini-1.0-pro output token limit is 2048
print(model.count_tokens(final))

total_tokens: 2289



## 後續步驟

我們希望你喜歡這個筆記本！作為練習，你可以嘗試調整延續提示，以採取人機合作的輸入來引導敍述。
